In [1]:
import math
import os
import random
import itertools

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython.display import clear_output
from auton_survival import datasets, preprocessing, models
from sklearn.model_selection import train_test_split

import torch
from torch import nn, matmul
from torch.nn.functional import softmax

clear_output()

In [2]:
# define the setup parameters
import matplotlib.pyplot as plt

from SurvTRACE.survtrace.dataset import load_data
from SurvTRACE.survtrace.evaluate_utils import Evaluator
from SurvTRACE.survtrace.utils import set_random_seed
from SurvTRACE.survtrace.model import SurvTraceSingle
from SurvTRACE.survtrace.train_utils import Trainer
from SurvTRACE.survtrace.config import STConfig

STConfig['data'] = 'metabric'
# STConfig['duration_index'] = [0.11225496, 2.06987187, 3.72357902, 6.68738364, 14.30566203]


set_random_seed(STConfig['seed'])

hparams = {
    'batch_size': 2,
    'weight_decay': 1e-4,
    'learning_rate': 1e-3,
    'epochs': 1,
}

In [3]:
df = pd.read_csv('/Users/davidlee/Downloads/PBC2_embedded.csv')

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,event,duration
0,-0.640790,8.601443,-0.821111,-0.821115,-0.821100,-0.821370,-0.821102,-0.821252,-0.821242,-0.821124,...,-0.821124,-0.821170,-0.821111,-0.821121,-0.821300,-0.821092,-0.821129,-0.821197,1.0,1.095170
1,2.690124,6.514240,-1.052684,-1.052686,-1.052673,-1.052895,-1.052674,-1.052777,-1.052789,-1.052690,...,-1.052690,-1.052723,-1.052681,-1.052690,-1.052828,-1.052666,-1.052695,-1.052747,0.0,14.152338
2,0.362566,7.969614,-0.890715,-0.890719,-0.890704,-0.890959,-0.890706,-0.890842,-0.890838,-0.890726,...,-0.890726,-0.890769,-0.890714,-0.890724,-0.890891,-0.890697,-0.890731,-0.890795,1.0,2.770781
3,1.734989,7.121678,-0.987260,-0.987262,-0.987248,-0.987484,-0.987250,-0.987367,-0.987372,-0.987268,...,-0.987267,-0.987305,-0.987257,-0.987267,-0.987417,-0.987242,-0.987273,-0.987329,1.0,5.270507
4,1.305060,7.377329,-0.955973,-0.955976,-0.955962,-0.956203,-0.955964,-0.956086,-0.956089,-0.955982,...,-0.955982,-0.956021,-0.955971,-0.955980,-0.956136,-0.955955,-0.955987,-0.956046,0.0,4.120578


In [5]:
import pandas as pd
from SurvTRACE.survtrace.utils import LabelTransform
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder, StandardScaler

In [6]:
import numpy as np
from SurvTRACE.survtrace.config import STConfig
def test_pbc2(df):
  get_target = lambda df: (df['duration'].values, df['event'].values)
  horizons = [.25, .5, .75]
  times = np.quantile(df["duration"][df["event"]==1.0], horizons).tolist()

  # cols_standardize = ["seq_temporal_SGOT", "seq_temporal_age", "seq_temporal_albumin", "seq_temporal_alkaline", "seq_temporal_platelets", "seq_temporal_prothrombin", "seq_temporal_serBilir", "seq_temporal_serChol"]
  # cols_categorical = ["seq_static_sex", "seq_temporal_ascites", "seq_temporal_drug", "seq_temporal_edema", "seq_temporal_hepatomegaly", "seq_temporal_histologic", "seq_temporal_spiders"]

  # cols_standardize = ["seq_temporal_SGOT", "seq_temporal_age", "seq_temporal_albumin", "seq_temporal_alkaline", "seq_temporal_platelets", "seq_temporal_prothrombin", "seq_temporal_serBilir", "seq_temporal_serChol"]
  # cols_categorical = ['seq_static_sex_1.0', 'seq_temporal_ascites_1.0', 'seq_temporal_ascites_2.0', 'seq_temporal_drug_1.0', 'seq_temporal_edema_1.0', 'seq_temporal_edema_2.0', 'seq_temporal_hepatomegaly_1.0', 'seq_temporal_hepatomegaly_2.0', 'seq_temporal_histologic_1.0', 'seq_temporal_histologic_2.0', 'seq_temporal_histologic_3.0', 'seq_temporal_spiders_1.0', 'seq_temporal_spiders_2.0']

  cols_standardize = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
  cols_categorical = []

  df_feat = df.drop(["duration","event"],axis=1)
  df_feat_standardize = df_feat[cols_standardize]
  df_feat_standardize_disc = StandardScaler().fit_transform(df_feat_standardize)
  df_feat_standardize_disc = pd.DataFrame(df_feat_standardize_disc, columns=cols_standardize)

  # must be categorical feature ahead of numerical features!
  df_feat = pd.concat([df_feat[cols_categorical], df_feat_standardize_disc], axis=1)

  vocab_size = 0
  for _,feat in enumerate(cols_categorical):
      df_feat[feat] = LabelEncoder().fit_transform(df_feat[feat]).astype(float) + vocab_size
      vocab_size = df_feat[feat].max() + 1
  print(vocab_size)

  # get the largest duraiton time
  max_duration_idx = df["duration"].argmax()
  # df_test = df_feat.drop(max_duration_idx).sample(frac=0.2)
  # df_train = df_feat.drop(df_test.index)
  # df_val = df_train.drop(max_duration_idx).sample(frac=0.2)
  # df_train = df_train.drop(df_val.index)


  # # Hung edited here
  df_train = df_feat.iloc[125:] #
  df_train = df_train.sample(frac=1)

  df_remaining = df_feat.iloc[:124] #
  df_remaining = df_remaining.sample(frac=1)

  # Now let's say you want to split the remaining data into test and validation sets
  df_test = df_remaining.sample(frac=0.5) # 20% of the training data for testing
  df_val = df_remaining.drop(df_test.index) # 20% of the training data for validating


  # assign cuts
  labtrans = LabelTransform(cuts=np.array([df["duration"].min()]+times+[df["duration"].max()]))
  labtrans.fit(*get_target(df.loc[df_train.index]))
  y = labtrans.transform(*get_target(df)) # y = (discrete duration, event indicator)
  df_y_train = pd.DataFrame({"duration": y[0][df_train.index], "event": y[1][df_train.index], "proportion": y[2][df_train.index]}, index=df_train.index)
  df_y_val = pd.DataFrame({"duration": y[0][df_val.index], "event": y[1][df_val.index],  "proportion": y[2][df_val.index]}, index=df_val.index)
  # df_y_test = pd.DataFrame({"duration": y[0][df_test.index], "event": y[1][df_test.index],  "proportion": y[2][df_test.index]}, index=df_test.index)
  df_y_test = pd.DataFrame({"duration": df['duration'].loc[df_test.index], "event": df['event'].loc[df_test.index]})
  STConfig['labtrans'] = labtrans
  STConfig['num_numerical_feature'] = int(len(cols_standardize))
  STConfig['num_categorical_feature'] = int(len(cols_categorical))
  STConfig['num_feature'] = int(len(df_train.columns))
  STConfig['vocab_size'] = int(vocab_size)
  STConfig['duration_index'] = labtrans.cuts
  STConfig['out_feature'] = int(labtrans.out_features)
  print("______")
  print(int(len(cols_standardize)))
  print("______")
  print(int(len(cols_categorical)))
  print("______")
  print(int(len(df_train.columns)))
  print("______")
  print(int(vocab_size))
  print("______")
  print(labtrans.cuts)
  print("______")
  print(int(labtrans.out_features))

  return y, df, df_train, df_y_train, df_test, df_y_test, df_val, df_y_val

In [7]:
y, df, df_train, df_y_train, df_test, df_y_test, df_val, df_y_val = test_pbc2(df)

0
______
21
______
0
______
21
______
0
______
[ 0.11225496  2.06987187  3.72357902  6.68738364 14.30566203]
______
4


/Users/davidlee/Documents/GitHub/Surtimesurvival/SurvTRACE/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")


In [8]:
# get model
model = SurvTraceSingle(STConfig)

# initialize a trainer
trainer = Trainer(model)
train_loss, val_loss = trainer.fit((df_train, df_y_train), (df_val, df_y_val),
        batch_size=16,
        epochs=hparams['epochs'],
        learning_rate=hparams['learning_rate'],
        weight_decay=hparams['weight_decay'],)

GPU not found! will use cpu for training!
train with single event
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([16, 0])
torch.Size([16, 21])
torch.Size([11, 0])
torch.Size([11, 21])
[Train-0]: 21.360479593276978
[Val-0]: 1.9090708494186401


/Users/davidlee/Documents/GitHub/Surtimesurvival/SurvTRACE/survtrace/train_utils.py:216: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
